In [1]:
from ultralytics import YOLO
from ensemble_boxes import weighted_boxes_fusion

In [2]:
# pt_model1 = YOLO("runs/detect/yolov9e 0.995 0.825/weights/epoch62.pt")  # load a pretrained model (recommended for training)
# pt_model2 = YOLO("../yolov9c_0.99_0.769.pt")  # load a pretrained model (recommended for training)
# pt_model2 = YOLO("runs/detect/yolov9e 0.995 0.825/weights/epoch65.engine")  # load a pretrained model (recommended for training)
pt_model2 = YOLO("../yolov9e_0.995_0.823_epoch65.pt")  # load a pretrained model (recommended for training)

In [4]:
pt_model2.export(format="engine", imgsz=(896, 768), simplify=True, workspace=8, batch=6, half=True, dynamic=False)

WARNING  TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.2.18  Python-3.9.13 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv9e summary (fused): 687 layers, 57377171 parameters, 0 gradients, 189.1 GFLOPs

PyTorch: starting from '..\yolov9e_0.995_0.823_epoch65.pt' with input shape (6, 3, 896, 768) BCHW and output shape(s) (6, 5, 14112) (334.6 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success  10.9s, saved as '..\yolov9e_0.995_0.823_epoch65.onnx' (219.3 MB)

TensorRT: starting export with TensorRT 10.0.1...
TensorRT: input "images" with shape(6, 3, 896, 768) DataType.FLOAT
TensorRT: output "output0" with shape(6, 5, 14112) DataType.FLOAT
TensorRT: building FP16 engine as ..\yolov9e_0.995_0.823_epoch65.engine
TensorRT: export success  360.4s, saved as '..\yolov9e_0.995_0.823_epoch65.engine' (115.1 MB)

Export complete (362.7s)
Results saved to C:\Users\alien\Documents

'..\\yolov9e_0.995_0.823_epoch65.engine'

In [4]:
pt_model2.export(format="engine", imgsz=(1600, 1600), simplify=True, workspace=8, batch=1, half=True, dynamic=False)

WARNING  TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.2.18  Python-3.9.13 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
YOLOv9e summary (fused): 687 layers, 57377171 parameters, 0 gradients, 189.1 GFLOPs

PyTorch: starting from '..\yolov9e_0.995_0.823_epoch65.pt' with input shape (1, 3, 1600, 1600) BCHW and output shape(s) (1, 5, 52500) (334.6 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success  11.3s, saved as '..\yolov9e_0.995_0.823_epoch65.onnx' (219.7 MB)

TensorRT: starting export with TensorRT 10.0.1...
TensorRT: input "images" with shape(1, 3, 1600, 1600) DataType.FLOAT
TensorRT: output "output0" with shape(1, 5, 52500) DataType.FLOAT
TensorRT: building FP16 engine as ..\yolov9e_0.995_0.823_epoch65.engine
TensorRT: export success  325.8s, saved as '..\yolov9e_0.995_0.823_epoch65.engine' (116.2 MB)

Export complete (327.2s)
Results saved to C:\Users\alien\Docum

'..\\yolov9e_0.995_0.823_epoch65.engine'

In [4]:
from PIL import Image, ImageDraw
import numpy as np
im1 = Image.open('../../../data/images/image_5095.jpg')
im2 = Image.open('../../../data/images/image_5100.jpg')

In [10]:
from PIL import Image, ImageDraw
import numpy as np
im1 = Image.open('2.jpg')
im2 = Image.open('3.jpg')

In [4]:
result1 = pt_model1.predict([im1, im2], imgsz=1600, conf=0.3, iou=0.1, max_det=10)  # max F1, try augment=True and iou
result1_n = [(r.boxes.xyxyn.tolist(), r.boxes.conf.tolist()) for r in result1]
result1_n = [tuple(zip(*r)) for r in result1_n]
result1 = [(r.boxes.xyxy.tolist(), r.boxes.conf.tolist()) for r in result1]
result1 = [tuple(zip(*r)) for r in result1]  # list of tuple[box, conf] in each image in xyxy format

C:\Program Files\Python39\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


0: 928x1600 7 targets, 124.4ms
1: 928x1600 7 targets, 124.4ms
Speed: 7.6ms preprocess, 124.4ms inference, 932.8ms postprocess per image at shape (1, 3, 928, 1600)


In [1]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

06/06/2024 00:50:30 - INFO - numexpr.utils -   Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
06/06/2024 00:50:30 - INFO - numexpr.utils -   NumExpr defaulting to 8 threads.
06/06/2024 00:50:32 - INFO - albumentations.check_version -   A new version of Albumentations is available: 1.4.8 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


In [2]:
model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path='../yolov9e_0.995_0.823_epoch65.pt',
    confidence_threshold=0.5,
    image_size=896,
    standard_pred_image_size=1600,  # not used for TRT as it dont support standard pred
    device="cuda",
    cfg={"task": 'detect', "names": {'0': 'target'}, "imgsz": (896, 768), "half": True}
)

In [6]:
result = get_sliced_prediction(im1, model, perform_standard_pred=False, postprocess_class_agnostic=True, batch=6).object_prediction_list
result_n = [([r.bbox.minx / 1520, r.bbox.miny / 870, r.bbox.maxx / 1520, r.bbox.maxy / 870], r.score.value) for r in result]
result = [([r.bbox.minx, r.bbox.miny, r.bbox.maxx, r.bbox.maxy], r.score.value) for r in result]

Performing prediction on 6 slices.


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
pt_model2.predict([Image.new('RGB', (870, 760)) for i in range(6)], imgsz=896, conf=0.5, iou=0.1, max_det=10)


0: 800x896 (no detections), 26.0ms
1: 800x896 (no detections), 26.0ms
2: 800x896 (no detections), 26.0ms
3: 800x896 (no detections), 26.0ms
4: 800x896 (no detections), 26.0ms
5: 800x896 (no detections), 26.0ms
Speed: 3.8ms preprocess, 26.0ms inference, 0.2ms postprocess per image at shape (1, 3, 800, 896)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'target'}
 obb: None
 orig_img: array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
   

In [11]:
result2 = pt_model2.predict([im1, im2], imgsz=1600, conf=0.3, iou=0.1, max_det=10)  # max F1, try augment=True and iou
result2_n = [(r.boxes.xyxyn.tolist(), r.boxes.conf.tolist()) for r in result2]
result2_n = [tuple(zip(*r)) for r in result2_n]
result2 = [(r.boxes.xyxy.tolist(), r.boxes.conf.tolist()) for r in result2]
result2 = [tuple(zip(*r)) for r in result2]  # list of tuple[box, conf] in each image in xyxy format


0: 928x1600 4 targets, 58.5ms
1: 928x1600 3 targets, 58.5ms
Speed: 8.0ms preprocess, 58.5ms inference, 2.0ms postprocess per image at shape (1, 3, 928, 1600)


In [27]:
result2 = pt_model2.predict([im1], imgsz=(1536, 896), conf=0.3, iou=0.1, max_det=10)  # max F1, try augment=True and iou
result2_n = [(r.boxes.xyxyn.tolist(), r.boxes.conf.tolist()) for r in result2]
result2_n = [tuple(zip(*r)) for r in result2_n]
result2 = [(r.boxes.xyxy.tolist(), r.boxes.conf.tolist()) for r in result2]
result2 = [tuple(zip(*r)) for r in result2]  # list of tuple[box, conf] in each image in xyxy format


0: 1536x896 7 targets, 45.0ms
Speed: 8.0ms preprocess, 45.0ms inference, 2.0ms postprocess per image at shape (1, 3, 1536, 896)


In [7]:
boxes_list = [[r[0] for r in result1_n[0]], [r[0] for r in result2_n[0]]]
scores_list = [[r[1] for r in result1_n[0]], [r[1] for r in result2_n[0]]]
labels_list = [[0] * len(result1_n[0]), [0] * len(result2_n[0])]
weights = [0.1, 1]
boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=0.5, skip_box_thr=1e-4)
boxes = boxes.tolist()
# normalize
boxes = [[x1*1520, y1*870, x2*1520, y2*870] for x1, y1, x2, y2 in boxes]
scores = scores.tolist()
wbf_result = [list(zip(boxes, scores))]

In [13]:
# plot
for im, boxes in zip([im1, im2], result2):
    im = im.copy()
    draw = ImageDraw.Draw(im)
    for (x1, y1, x2, y2), conf in boxes:
        # print(x1, y1, x2, y2, conf)
        print([x1, y1, x2 - x1, y2 - y1], conf)
        draw.rectangle(xy=((x1, y1), (x2, y2)), outline='red')
        draw.text((x1, y1), text=f'{conf:.2f}', fill='red')
    im.show()

[805.4537963867188, 266.6635437011719, 65.88726806640625, 51.044525146484375] 0.8840249180793762
[780.4070434570312, 344.8293762207031, 60.640869140625, 30.552337646484375] 0.8493534326553345
[247.44850158691406, 504.761474609375, 110.45524597167969, 38.27239990234375] 0.8326224684715271
[1415.942138671875, 158.29652404785156, 104.057861328125, 83.5587158203125] 0.797531008720398
[926.0911865234375, 295.0468444824219, 114.6827392578125, 57.679962158203125] 0.8768801093101501
[1131.968994140625, 235.7434844970703, 57.12158203125, 48.26167297363281] 0.8663981556892395
[859.451904296875, 518.83251953125, 67.2882080078125, 43.86517333984375] 0.8398226499557495
